# Temple Expert RAG System - Testing Notebook

This notebook tests the complete RAG system:
- Fine-tuned Llama-3 model (from Hugging Face)
- Tavily AI search (real-time information)
- Intelligent query routing

**Model**: `Karpagadevi/llama-3-temple-expert`

## Setup: Clone Repository & Install Dependencies

In [ ]:
# Clone the repository
!git clone https://github.com/karpagadevip-droid/temple_llm_model.git
%cd temple_llm_model

# List files to verify
!ls -la

In [ ]:
# Install required packages
!pip install -q unsloth transformers accelerate bitsandbytes python-dotenv tavily-python

## Configuration: Set API Keys in .env File

In [ ]:
%%writefile .env
# Tavily AI API Key (get from https://tavily.com/)
TAVILY_API_KEY=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

# Hugging Face Token (optional, for private models)
HUGGINGFACE_TOKEN=hf_your_token_here

# Model to test
HUGGINGFACE_MODEL_PATH=Karpagadevi/llama-3-temple-expert

In [ ]:
# Verify .env file created
!cat .env

## Test 1: Tavily Search Only (Fast Test)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from tavily_search import TavilySearcher

print("Testing Tavily AI Search...")
print("=" * 70)

searcher = TavilySearcher()
results = searcher.search_temple_tickets("Meenakshi Temple")

if results['success']:
    print("\n✅ Search successful!")
    print(f"\nAI Summary:\n{results['answer']}")
    print(f"\nSources found: {len(results['results'])}")
    
    # Show first source
    if results['results']:
        first = results['results'][0]
        print(f"\nTop Result:")
        print(f"  Title: {first['title']}")
        print(f"  Relevance: {first['score']:.2f}")
        print(f"  URL: {first['url']}")
else:
    print(f"\n❌ Search failed: {results.get('error')}")
    print("\nCheck your TAVILY_API_KEY in .env file")

## Test 2: Load Fine-Tuned Model

In [ ]:
from model_loader import TempleModelLoader
import os

print("Loading fine-tuned model from Hugging Face...")
print("=" * 70)
print(f"Model: {os.getenv('HUGGINGFACE_MODEL_PATH')}")
print("\nThis may take 2-3 minutes on first run (downloads ~4GB)...")
print()

loader = TempleModelLoader()
model, tokenizer = loader.load_model()

print("\n✅ Model loaded successfully!")
print("\nTesting with a query...")

response = loader.generate_response("Tell me about Meenakshi Temple", max_length=256)
print(f"\nModel Response:\n{response}")

## Test 3: Complete RAG System (Model + Search)

In [ ]:
from rag_orchestrator import TempleRAG

print("=" * 70)
print("Initializing Complete RAG System")
print("=" * 70)
print()

# Initialize RAG with model
rag = TempleRAG(
    load_model=True,
    model_name=os.getenv('HUGGINGFACE_MODEL_PATH')
)

print("\n✅ RAG system ready!\n")

## Test 4: Query Classification & Routing

In [ ]:
# Test different query types
test_queries = [
    {
        'query': "What is the ticket price for Meenakshi Temple?",
        'expected': 'search',
        'explanation': 'Real-time info → uses Tavily search'
    },
    {
        'query': "Tell me about the history of Meenakshi Temple",
        'expected': 'model',
        'explanation': 'Historical info → uses fine-tuned model'
    },
    {
        'query': "Tell me about Meenakshi Temple and how to visit",
        'expected': 'hybrid',
        'explanation': 'Both needed → uses model + search'
    }
]

for i, item in enumerate(test_queries, 1):
    print("\n" + "=" * 70)
    print(f"DEMO {i}/3: {item['explanation']}")
    print("=" * 70)
    print()
    
    result = rag.generate_response(item['query'])
    
    print(f"Strategy: {result['strategy']} (expected: {item['expected']})")
    print(f"Source: {result['source']}")
    print(f"\nResponse (first 400 chars):")
    response = result['response']
    print(response[:400] + "..." if len(response) > 400 else response)
    print()

## Test 5: Usage Statistics

In [ ]:
stats = rag.get_stats()
tavily_stats = stats['tavily_usage']

print("\n" + "=" * 70)
print("USAGE STATISTICS")
print("=" * 70)
print(f"Tavily Searches Used: {tavily_stats['searches_used']}/{tavily_stats['free_tier_limit']}")
print(f"Remaining: {tavily_stats['remaining']} ({100-tavily_stats['percentage_used']:.1f}%)")
print(f"Model Loaded: {stats['model_loaded']}")
print("=" * 70)

print("\n✅ All tests complete!")

## Test 6: Interactive Testing (Try Your Own Queries)

In [ ]:
# Try your own query!
query = input("Ask about a temple: ")
print()

result = rag.generate_response(query)

print(f"\n{'='*70}")
print(f"Strategy: {result['strategy']}")
print(f"Source: {result['source']}")
print(f"\nResponse:\n{result['response']}")
print('='*70)

## Test 7: Compare 60-step vs 600-step Models

Run this after your 600-step model finishes training!

In [ ]:
# Uncomment and run after 600-step model is ready

# test_query = "Tell me about Meenakshi Temple"

# print("Loading 60-step model...")
# rag_60 = TempleRAG(load_model=True, model_name="Karpagadevi/llama-3-temple-expert")
# result_60 = rag_60.generate_response(test_query)

# print("\nLoading 600-step model...")
# rag_600 = TempleRAG(load_model=True, model_name="Karpagadevi/llama-3-temple-expert-600")
# result_600 = rag_600.generate_response(test_query)

# # Compare
# print("\n" + "="*70)
# print("60-STEP MODEL RESPONSE:")
# print("="*70)
# print(result_60['response'])

# print("\n" + "="*70)
# print("600-STEP MODEL RESPONSE:")
# print("="*70)
# print(result_600['response'])

# print("\n" + "="*70)
# print("Which one is better? 👆")
# print("="*70)